In [ ]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [ ]:
prompt = """
<human>: Singapore Chicken Wings
<assistant>:
""".strip()

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Singapore Chicken Wings
<assistant>: 1. Preheat oven to 400°F. 2. In a bowl, mix together 1/4 cup olive oil, 1/4 cup soy sauce, 1/4 cup honey, 1/4 cup garlic powder, 1/4 cup paprika, 1/4 cup cayenne pepper, and 1/4 cup water. 3. Place chicken wings in a baking dish and pour the marinade over them. 4. Bake for 30 minutes, turning the wings once halfway through. 5. Brush the wings with the remaining marinade and bake for an additional 10 minutes. 6. Enjoy!
CPU times: user 51.4 s, sys: 39.9 ms, total: 51.5 s
Wall time: 54 s


# Prep dataset

In [ ]:
#choose the dataset you would like to train with
data = load_dataset("csv", data_files="/content/drive/MyDrive/GCPLP/testforkb_LLM.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-215145fd0f1c5467/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['Ingredients', 'Recipe'],
        num_rows: 107
    })
})

In [ ]:
data["train"][0]

{'Ingredients': 'package chow mein noodle large cook frozen shrimp egg bean sprout water chestnut baby sweet corn cob small onion medium tomato clove garlic mince fresh ginger mince curry powder peanut oil chicken broth stock soy sauce sugar chili oil chinese',
 'Recipe': " Singapore Fried Noodles   Steps:  1) PREPARATION. 2) Boil noodles until cooked but still firm- 3 minutes. 3) Drain and rinse with cold water, then refrigerate. 4) Thaw shrimp and cut off tails. 5) Beat eggs and stir fry/scramble. 6) Set aside. 7) Blanch bean sprouts, water chestnuts and corn. 8) Set aside. 9) Slice onion and set aside. 10) Mix sauce ingredients in bowl and set aside. 11) STIR FRY. 12) Add 2 tbsp peanut oil to hot wok. 13) When oil begins to smoke, toss in garlic, ginger, shrimp& blanched vegetables. 14) Stir-fry on high heat for 2 minutes. 15) Add sauce and continue cooking about three minutes. 16) Remove to holding bowl. 17) Rinse wok and return to high heat. 18) When it is dry, reduce heat to medi

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["Ingredients"]}
<assistant>: {data_point["Recipe"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/107 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['Ingredients', 'Recipe', 'input_ids', 'attention_mask'],
    num_rows: 107
})

# Finetune the model

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.301500
2,2.127500
3,2.334700
4,2.166500
5,2.489800
6,1.959800
7,2.191800
8,2.307600
9,2.226400
10,2.080700


TrainOutput(global_step=26, training_loss=2.0799446472754846, metrics={'train_runtime': 258.4592, 'train_samples_per_second': 0.414, 'train_steps_per_second': 0.101, 'total_flos': 845909392144128.0, 'train_loss': 2.0799446472754846, 'epoch': 0.97})

# Save trained model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "Sanjeven/ChefAndrew_SG2"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sanjeven/ChefAndrew_SG2/commit/992fc49e8211b6f5a135520c2d67b44f11867227', commit_message='Upload model', commit_description='', oid='992fc49e8211b6f5a135520c2d67b44f11867227', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

(…)rew_SG2/resolve/main/adapter_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

# Run the finetuned model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: you are a chef, give me a recipe, cooking instructions and ingredients for crab
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: you are a chef, give me a recipe, cooking instructions and ingredients for crab
<assistant>: Crab Linguine with White Wine and Lemon 
Ingredients: 
- 1 lb. linguine 
- 2 tablespoons olive oil 
- 2 cloves garlic, minced 
- 1/2 cup white wine 
- 2 tablespoons lemon juice 
- 1/2 cup heavy cream 
- 1/2 cup grated parmesan cheese 
- 1/2 cup chopped fresh parsley 
- 1/2 lb. crab meat, picked over 

Instructions: 
1. Cook linguine according to package instructions. 
2. Heat olive oil in a large skillet over medium-high heat. Add garlic and cook for 1 minute. 
3. Add white wine and lemon juice and bring to a boil. Reduce heat to low and simmer for 2 minutes. 
4. Drain linguine and return to pot. 
5. Add cream, parmesan cheese, and parsley and mix until combined. 
6. Add crab meat and mix
CPU times: user 1min 18s, sys: 71.2 ms, total: 1min 18s
Wall time: 1min 19s
